# Funciones Hash Criptograficas

## Funcion Hash

Es una unción que toma como entrada un string de largo variable y devuelve como salida un string de largo fijo.

*Nota: podemos reemplazar string por secuencia de bytes o bits en la definicion para hacerla mas generica.*

Para que una funcion hash sea util debe ser fácil de calcular por un ordenador, es decir, no debe tomar mucho tiempo poder calcular su salida.

A continuacion un ejemplo de funcion hash que recibe un string

In [ ]:
def naive_hash(s):
    """
    Suma los valores ASCII de cada caracter y toma el modulo.

    :param str s: La string a ser hasheada
    :return: El hash en hexadecimal con 4 caracteres
    :rtype: string
    """
    mod = 10000
    hash = sum(ord(char) for char in s) % mod
    return f'{hash:04x}'

# Ejemplos
inputs = ["Hola", "hola", "olah", "Hola!", "Hola Mundo", "Holaaaaa", "Chau"]
hashes = {s: naive_hash(s) for s in inputs}

hashes


{'Hola': '0184',
 'hola': '01a4',
 'olah': '01a4',
 'Hola!': '01a5',
 'Hola Mundo': '03a7',
 'Holaaaaa': '0308',
 'Chau': '0181'}


## Funcion hash criptografica

Para que la funcion hash sea criptografica debe cumplir las siguiente propiedades de seguridad:
- **Pre-image resistance**
- **Second pre-image resistance**
- **Collision resistance**

Una funcion que cumple estas propiedades es por ejemplo [SHA256](https://emn178.github.io/online-tools/sha256.html) que es la utilizada en bitcoin.

### Pre-image resistance

**Dado:** `hash(m1)`

**Es dificil de encontrar:** `m2` tal que  `hash(m2) == hash(m1)`

#### Ejercicio 1a:

Dado: `naive_hash(m1) == 049e`

Mostrar que la funcion creada anteriormente `naive_hash()` no cumple con la condicion pre-image resistance

In [ ]:
#@title Solucion Ejercicio 1a {display-mode:"form"}

#naive_hash("helicoptero") == '049e'
decimal = int('049e', 16)
print(decimal)
# 'd' == 100 y 'R' == 82
m2 = 'd' * 11 + 'R'
print(naive_hash(m2))

1182
049e


#### Ejercicio 1b:

Encontrar palabras en castellano que colisionen con el Hash `049e` de la funcion `naive_hash()`

Ayuda: https://github.com/webpwnized/byepass/blob/master/dictionaries/top-10000-spanish-words.txt

### Second pre-image resistance

**Dado:** `m1`

**Es dificil encontrar:** `m2` donde `m1 != m2` tal que `hash(m1) = hash(m2)`

#### Ejercicio 2:

Dado: `m1='Hola'`

Mostrar que la funcion creada anteriormente `naive_hash()` no cumple con la condicion Second pre-image resistance

In [ ]:
#@title Solucion Ejercicio 2 {display-mode:"form"}

print("Hola", "=", naive_hash("Hola"))
print("olaH", "=", naive_hash("olaH"))
print("aloH", "=", naive_hash("aloH"))
print("Halo", "=", naive_hash("aloH"))

Hola = 0184
olaH = 0184
aloH = 0184
Halo = 0184


### Collision resistance

**Es dificil encontrar:** `m1` y `m2` donde `m1 != m2` tal que `hash(m1) = hash(m2)`

#### Ejercicio:

Utilizar la libreria [hashlib](https://docs.python.org/3/library/hashlib.html) para crear el hash de tu nombre utilizando la funcion sha256.

El resultado del hash sera en bytes utiliza distintas codificaciones para mostrarlo. Por ejemplo hexadecimal, base16, base32 y base64.

In [ ]:
import hashlib
x = hashlib.sha256(b"Alicia")
print(f'{"Hash en bytes:":<21}{x.digest()}')
print(f'{"Hash en hexadecimal:":<21}{x.hexdigest()}')
import base64
print(f'{"Hash en base16:":<21}{base64.b16encode(x.digest())}')
print(f'{"Hash en base32:":<21}{base64.b32encode(x.digest())}')
print(f'{"Hash en base64:":<21}{base64.b64encode(x.digest())}')

Hash en bytes:       b'*O\x07\x9d,;\xd9y\xaeQ\x9d\xc0\x9f\xdb\xe9\xb7\xef;\x919\x96\xa0\xb5\xd9p\xab5\xab\xe8\x95"O'
Hash en hexadecimal: 2a4f079d2c3bd979ae519dc09fdbe9b7ef3b913996a0b5d970ab35abe895224f
Hash en base16:      b'2A4F079D2C3BD979AE519DC09FDBE9B7EF3B913996A0B5D970AB35ABE895224F'
Hash en base32:      b'FJHQPHJMHPMXTLSRTXAJ7W7JW7XTXEJZS2QLLWLQVM22X2EVEJHQ===='
Hash en base64:      b'Kk8HnSw72XmuUZ3An9vpt+87kTmWoLXZcKs1q+iVIk8='


## Uso de estas propiedades en Bitcoin

Las funciones hash son muy utilizadas en las cadenas de bloques (blockchains) como Bitcoin y otras similares.

Para que se utilizan?
- Relacionar cada bloque de la blockchain con el siguiente bloque
- Para el puzzle del proof of work
- Para la estructura de merkle tree de las transacciones

El las siguientes unidades veremos estos puntos en detalle.

#### Ejercicio E1:

- Supongamos que tenemos un dado de seis lados y queremos hashear su resultado.
- Tenemos 6 hashes, uno para cada cara del dado. m puede ser: `["uno","dos","tres","cuatro","cinco","seis"]`
- Utilizo `naive_hash(s)`

Si sabemos que el resultado de una tirada del dado es 028e, adivinar que cara del dado salió

In [ ]:
#@title Solucion Ejercicio E1 {display-mode:"form"}

for item in ["uno","dos","tres","cuatro","cinco","seis"]:
  print(item,"=", naive_hash(item))

uno = 0152
dos = 0146
tres = 01be
cuatro = 028e
cinco = 020c
seis = 01b4


#### Ejercicio E2:

- Supongamos que tenemos un dado de seis lados y queremos hashear su resultado.
- Tenemos 6 hashes, uno para cada cara del dado. m puede ser: `["uno","dos","tres","cuatro","cinco","seis"]`
- Tenemos un [secreto r](https://en.wikipedia.org/wiki/Salt_(cryptography)) de 3 letras minusculas (`'abcdefghijklmnopqrstuvwxyz'`)
- Utilizo `naive_hash(s+r)` donde s+r es s concatenado con r

Si sabemos que el resultado de una tirada del dado es 0377, adivinar que cara del dado salió

In [ ]:
#@title Solucion Ejercicio E2 {display-mode:"form"}

ascii_lowercase = 'abcdefghijklmnopqrstuvwxyz'
for item in ["uno","dos","tres","cuatro","cinco","seis"]:
  for l1 in ascii_lowercase:
    for l2 in ascii_lowercase:
      for l3 in ascii_lowercase:
        salt = l1 + l2 + l3
        tmp = item + salt
        h = naive_hash(tmp)
        if h == "0377":
            print(item, salt,"=", h)

cinco wzz = 0377
cinco xyz = 0377
cinco xzy = 0377
cinco yxz = 0377
cinco yyy = 0377
cinco yzx = 0377
cinco zwz = 0377
cinco zxy = 0377
cinco zyx = 0377
cinco zzw = 0377


#### Ejercicio E3:

Usar la función hash SHA256 para crear una url única para una página web [Ayuda](https://developers.cloudflare.com/web3/ipfs-gateway/concepts/ipfs/#:~:text=Every%20file%20added%20to%20IPFS,used%20into%20a%20single%20string.)

In [ ]:
import hashlib
import requests
r = requests.get('https://www.freecodingtour.com/links.html')
x = hashlib.sha256(r.content)
print(x.digest())
print(x.hexdigest())

b"\x97\xfd\x8b.\x99s\x1a\x8f~\xf3gq\xa6\xb7+J\xb0\xd7fS\r\x1c\xb8k*\xd1\xfd\xca\xa5\xd1'\xac"
97fd8b2e99731a8f7ef36771a6b72b4ab0d766530d1cb86b2ad1fdcaa5d127ac


#### Ejercicio E4:

Utilizar un secreto (salt) para cifrar la respuesta a un puzzle, publicar nuestra prediccion hasheada en algun lugar publico (twitter) y luego los ganadores revelar su prediccion y secreto para comprobar quien gano.

## Como crear funciones hash?

La respuesta a esta pregunta sobrepasa el alcance de este curso, pero aqui dejamos algunos links de interes:

- https://ocw.mit.edu/courses/6-046j-design-and-analysis-of-algorithms-spring-2015/resources/lecture-21-cryptography-hash-functions/
- https://iacr.org/archive/fse2004/30170373/30170373.pdf
- https://en.wikipedia.org/w/index.php?title=SHA-1&oldid=1161401043#Example_hashes
- https://en.wikipedia.org/wiki/MD5#Pseudocode

### Si tuviesemos un ordenador cuantico podriamos encontrar colisiones mas rapido?

No mucho debido a que las funciones hash no son estructuradas:
- https://crypto.stackexchange.com/questions/44386/are-cryptographic-hash-functions-quantum-secure